<a href="https://colab.research.google.com/github/RochX/comp486-assignments/blob/main/assignment4/assignment4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 4
## Xavier Silva

## Part 1

### Question 1
Use `read_csv` in `pandas` to read the CSV (`adm_data.csv`) file and assign it to a variable and print it.

In [1]:
from pathlib import Path
import pandas as pd
import urllib.request

# download the data if it is not downloaded
if not Path("adm_data.csv").is_file():
  # here I download the data from my personal git repo for this class instead of using Google Drive
  url = "https://raw.githubusercontent.com/RochX/comp486-assignments/main/assignment2/adm_data.csv"
  urllib.request.urlretrieve(url, "adm_data.csv")

In [2]:
adm_data=pd.read_csv('adm_data.csv')
adm_data

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337,118,4,4.5,4.5,9.65,1,0.92
1,2,324,107,4,4.0,4.5,8.87,1,0.76
2,3,316,104,3,3.0,3.5,8.00,1,0.72
3,4,322,110,3,3.5,2.5,8.67,1,0.80
4,5,314,103,2,2.0,3.0,8.21,0,0.65
...,...,...,...,...,...,...,...,...,...
395,396,324,110,3,3.5,3.5,9.04,1,0.82
396,397,325,107,3,3.0,3.5,9.11,1,0.84
397,398,330,116,4,5.0,4.5,9.45,1,0.91
398,399,312,103,3,3.5,4.0,8.78,0,0.67


### Question 2
Split the data into a training set and a test set.

In [3]:
import numpy as np

def shuffle_and_split_data(data, test_ratio):
    shuffled_indices = np.random.permutation(len(data))
    test_set_size = int(len(data) * test_ratio)
    test_indices = shuffled_indices[:test_set_size]
    train_indices = shuffled_indices[test_set_size:]
    return data.iloc[train_indices], data.iloc[test_indices]

In [4]:
from sklearn.model_selection import train_test_split
np.random.seed(42)

adm_training_set, adm_test_set  = shuffle_and_split_data(adm_data, 0.15)
len(adm_training_set), len(adm_test_set)

(340, 60)

### Question 3
Using the training set, train a linear regression model to predict the chance of Admission to a university using `LinearRegression()` in Scikit-Learn, singular value
decomposition (SVD), Stochastic Gradient Descent, and Polynomial Regression with degree 2, then fill out the following table (show your full work). For each model:
1. What are the values of the model parameters ($\Theta$'s)?
2. Predict $\hat{y}$ for the new instance (`X_new`) where `X_new =[324, 112, 3, 2, 4.5, 8,1]`.
3. Calculate the score (using the test set).
4. Calculate the cost (`mean_squared_error`) (using the test set).
5. Write your observation about the results from the table.

*See the text cell with the table for all answers.*

In [5]:
# scale the training data
from sklearn.preprocessing import MinMaxScaler
feature_scaler = MinMaxScaler(feature_range=(0, 1)).set_output(transform="pandas")
label_scaler = MinMaxScaler(feature_range=(0, 1)).set_output(transform="pandas")

adm_training_features = feature_scaler.fit_transform(adm_training_set.drop(["Serial No.", "Chance of Admit "], axis=1))
adm_training_labels = label_scaler.fit_transform(adm_training_set[["Chance of Admit "]])

We use the same scaler on the test data.
Notice that the minimums are not consistently `0` and the maximums are not consistently `1`.
This tells us we didn't actually use the `fit_transform` function on the test set and thus has the same scaling as the training set.

In [6]:
# scale the test features
adm_test_features = feature_scaler.transform(adm_test_set.drop(["Serial No.", "Chance of Admit "], axis=1))
adm_test_features.describe()

,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research
count,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000
mean,0.545000,0.554167,0.554167,0.627083,0.620833,0.595833,0.583333
std,0.250684,0.225637,0.284082,0.234127,0.233533,0.211935,0.497167
min,0.120000,0.142857,0.250000,0.125000,0.125000,0.192308,0.000000
25%,0.360000,0.419643,0.250000,0.500000,0.500000,0.448718,0.000000
50%,0.580000,0.535714,0.500000,0.625000,0.625000,0.570513,1.000000
75%,0.700000,0.714286,0.750000,0.750000,0.750000,0.776442,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,0.967949,1.000000


In [7]:
# scale the test labels
adm_test_labels = label_scaler.transform(adm_test_set[["Chance of Admit "]])
adm_test_labels.describe()

,Chance of Admit
count,60.000000
mean,0.600000
std,0.269973
min,0.000000
25%,0.428571
50%,0.595238
75%,0.825397
max,0.984127


In [8]:
# set up the X_new variable with column names for each feature
X_new = [324, 112, 3, 2, 4.5, 8,1]
X_new = pd.DataFrame([X_new], columns=list(adm_training_features))
X_new = feature_scaler.transform(X_new)
X_new

,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research
0,0.68,0.714286,0.5,0.25,0.875,0.384615,1.0


#### `LinearRegression()`
Train the model, get coefficients and intercept.

In [9]:
# LinearRegression()
from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression()
lin_reg.fit(adm_training_features, adm_training_labels)
lin_reg.intercept_,lin_reg.coef_

(array([0.03120841]),
 array([[0.14984229, 0.12218902, 0.0405335 , 0.00439134, 0.10905379,
         0.56415567, 0.03652352]]))

Predict the chance of admission for the new instance.

In [10]:
# note that we have to undo the scaling
# we add ".reshape(-1, 1)" since undoing the scaling requires it
label_scaler.inverse_transform(lin_reg.predict(X_new).reshape(-1, 1))

array([[0.71212346]])

Calculate the score using the test set.

In [11]:
lin_reg.score(adm_test_features, adm_test_labels)

0.8102830881910719

Calculate the mean squared error.

In [12]:
from sklearn.metrics import mean_squared_error

adm_test_predictions = lin_reg.predict(adm_test_features)

lin_reg_mse = mean_squared_error(adm_test_labels, adm_test_predictions)
lin_reg_mse

0.013597095674771404

#### Singular Value Decomposition

Train the model, get coefficients and intercept.

WILL FINISH LATER: SEEMS TO GIVE SAME RESULTS AS `LinearRegression()`...

In [13]:
# Singular Value Decomposition
from sklearn.preprocessing import add_dummy_feature
adm_training_features_with_dummy = add_dummy_feature(adm_training_features)

theta_best_svd, residuals, rank, s = np.linalg.lstsq(adm_training_features_with_dummy, adm_training_labels, rcond=1e-6)
theta_best_svd

array([[0.03120841],
       [0.14984229],
       [0.12218902],
       [0.0405335 ],
       [0.00439134],
       [0.10905379],
       [0.56415567],
       [0.03652352]])

Predict the chance of admission for the new instance.

Calculate the score using the test set.

Calculate the mean squared error.

#### Stochastic Gradient Descent

Train the model, get coefficients and intercept.

In [14]:
# Stochastic Gradient Descent
from sklearn.linear_model import SGDRegressor

sgd_reg = SGDRegressor(max_iter=1000, tol=1e-5, penalty=None, eta0=0.01, n_iter_no_change=100, random_state=42)
sgd_reg.fit(adm_training_features, adm_training_labels)

sgd_reg.intercept_, sgd_reg.coef_

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


(array([0.07799905]),
 array([0.20518317, 0.17828782, 0.08993212, 0.06162137, 0.11685651,
        0.2646796 , 0.03967091]))

Predict the chance of admission for the new instance.

In [15]:
# note that we have to undo the scaling
# we add ".reshape(-1, 1)" since undoing the scaling requires it
label_scaler.inverse_transform(sgd_reg.predict(X_new).reshape(-1, 1))

array([[0.7488471]])

Calculate the score using the test set.

In [16]:
sgd_reg.score(adm_test_features, adm_test_labels)

0.7818493134990467

Calculate the mean squared error.

In [17]:
from sklearn.metrics import mean_squared_error

adm_test_predictions = sgd_reg.predict(adm_test_features)

sgd_reg_mse = mean_squared_error(adm_test_labels, adm_test_predictions)
sgd_reg_mse

0.01563495698716583

#### Polynomial Regression with degree 2

Train the model, get coefficients and intercept.

In [18]:
# Polynomial Regression with degree 2
from sklearn.preprocessing import PolynomialFeatures
poly_features = PolynomialFeatures(degree=2, include_bias=False)

adm_training_features_poly = poly_features.fit_transform(adm_training_features)

poly_reg = LinearRegression()
poly_reg.fit(adm_training_features_poly, adm_training_labels)

poly_reg.intercept_, poly_reg.coef_

(array([0.02914502]),
 array([[ 0.2168003 ,  0.24854737, -0.16227261,  0.05727338,  0.14362016,
          0.59874864, -0.03495433,  0.02002601, -0.228508  ,  0.14116221,
          0.05204462,  0.23940147, -0.44481444,  0.00827988,  0.1311509 ,
         -0.04844039,  0.35528208, -0.20658764, -0.42156865,  0.07691971,
         -0.14763842,  0.60503182, -0.01408859, -0.02536459, -0.03105193,
         -0.54738908, -0.09102589,  0.29495698, -0.06570548,  0.34098534,
         -0.71475671, -0.0081984 ,  0.50845454,  0.20880843, -0.03495433]]))

Predict the chance of admission for the new instance.

In [19]:
# note that we have to undo the scaling
# we add ".reshape(-1, 1)" since undoing the scaling requires it
X_new_poly = poly_features.transform(X_new)
label_scaler.inverse_transform(poly_reg.predict(X_new_poly).reshape(-1, 1))

array([[0.7319162]])

Calculate the score using the test set.

In [20]:
# should I be using fit_transform or just transform?
adm_test_features_poly = poly_features.fit_transform(adm_test_features)
poly_reg.score(adm_test_features_poly, adm_test_labels)

0.8110556651788536

Calculate the mean squared error.

In [21]:
adm_test_predictions = poly_reg.predict(adm_test_features_poly)

poly_reg_mse = mean_squared_error(adm_test_labels, adm_test_predictions)
poly_reg_mse

0.013541724737521625

#### Table for question 3:
Note that $\theta_0$ is the intercept for all regressions.

All values for $\theta$ have been rounded to 3 decimal places.

| Model/Technique | Thetas $\Theta$'s | Predict `X_new` | Score | `mean_squared_error` |
| --- | --- | --- | --- | --- |
| Linear regression using Scikit-Learn | $\theta_0, \theta_1, \dots, \theta_7 = \{0.031, 0.150, 0.122, 0.041 , 0.004, 0.109, 0.564, 0.037\}$ | 0.71 | 0.81 | 0.0136 |
| Singular Value Decomposition (SVD) |
| Stochastic Gradient Descent | $\theta_0, \theta_1, \dots, \theta_7 = \{0.078, 0.205, 0.178, 0.090, 0.062, 0.117, 0.265 , 0.040\}$ | 0.74 | 0.78 | 0.0156
| Polynomial Regression with degree 2 | $\theta_0, \theta_1, \dots = \theta_{35} = \{0.029, 0.217,  0.249, -0.162,  0.057,  0.144,  0.599, -0.035,  0.02, -0.229,  0.141,  0.052,  0.239, -0.445,  0.008,  0.131, -0.048, 0.355, -0.207, -0.422,  0.077, -0.148,  0.605, -0.014, -0.025, -0.031, -0.547, -0.091,  0.295, -0.066,  0.341, -0.715, -0.008, 0.508,  0.209, -0.035\}$ | 0.73 | 0.811 | 0.0135 |

### Question 4
Which model would you adopt and why?

### Question 5
Train a decision tree model with `max_depth=2`, then predict the chance of Admission of `X_new`

In [22]:
# TODO

## Part 2

### Question 1
Download the rock-paper-scissors data.

In [23]:
from pathlib import Path
import pandas as pd
import urllib.request

# download the data if it is not downloaded
if not Path("rps-training-data.csv").is_file():
  # here I download the data from my personal git repo for this class instead of using Google Drive
  url = "https://raw.githubusercontent.com/RochX/comp486-assignments/main/assignment4/rps-training-data.csv"
  urllib.request.urlretrieve(url, "rps-training-data.csv")

# download the data if it is not downloaded
if not Path("rps-test-data.csv").is_file():
  # here I download the data from my personal git repo for this class instead of using Google Drive
  url = "https://raw.githubusercontent.com/RochX/comp486-assignments/main/assignment4/rps-test-data.csv"
  urllib.request.urlretrieve(url, "rps-test-data.csv")

Use `read_csv` in `pandas` to read the *training* set from the CSV ("Rock-Paper-
Scissor" training dataset) file and assign it to a variable and print it.

In [24]:
import pandas as pd

rps_training_data = pd.read_csv("rps-training-data.csv", header=None).rename(columns={2304: "RPS Label"})
rps_training_data

,0,1,2,3,4,5,6,7,8,9,...,2295,2296,2297,2298,2299,2300,2301,2302,2303,RPS Label
0,255,255,255,255,255,255,255,255,255,255,...,254,254,254,253,253,253,253,253,253,1
1,255,255,255,255,255,255,255,255,255,255,...,254,254,254,253,253,253,253,253,253,1
2,255,255,255,255,255,255,255,255,255,255,...,254,254,254,254,253,254,253,253,253,1
3,255,255,255,255,255,255,255,255,255,255,...,254,254,254,253,253,253,253,253,253,1
4,255,255,255,255,255,255,255,255,255,255,...,254,254,254,254,254,254,253,253,253,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2515,253,253,253,253,253,253,253,253,253,253,...,246,246,246,247,248,248,248,249,249,3
2516,253,253,253,253,253,253,253,253,253,253,...,245,246,246,247,247,248,248,249,249,3
2517,253,253,253,253,253,253,253,253,253,253,...,246,246,246,247,247,248,248,249,249,3
2518,253,253,253,253,253,253,253,253,253,253,...,245,246,246,247,247,248,248,249,249,3


### Question 2
Use `read_csv` in `pandas` to read the *test* set from the CSV ("Rock-Paper-Scissor"
test dataset) file and assign it to a variable and print it.

In [25]:
rps_test_data = pd.read_csv("rps-test-data.csv", header=None).rename(columns={2304: "RPS Label"})
rps_test_data

,0,1,2,3,4,5,6,7,8,9,...,2295,2296,2297,2298,2299,2300,2301,2302,2303,RPS Label
0,253,253,253,253,253,253,253,253,253,252,...,254,247,249,249,249,250,250,250,251,1
1,253,253,253,253,253,253,253,253,253,253,...,254,247,249,249,250,250,250,250,251,1
2,253,253,253,253,253,253,253,253,253,253,...,254,247,249,249,249,250,250,250,250,1
3,253,253,253,253,253,253,253,253,253,252,...,254,247,249,249,249,250,250,250,251,1
4,253,253,253,253,253,253,253,253,253,253,...,254,247,249,249,249,250,250,250,251,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
367,253,254,254,253,253,253,253,253,253,253,...,248,248,249,249,249,249,249,249,250,3
368,254,254,254,254,254,253,253,253,253,253,...,249,249,249,249,249,249,249,250,250,3
369,254,254,254,254,253,253,253,253,253,253,...,248,249,249,249,249,249,249,249,250,3
370,254,254,254,254,254,253,253,253,253,253,...,248,249,249,249,249,249,249,249,250,3


Shuffle the test and training data.

In [26]:
from sklearn.utils import shuffle

rps_training_data_shuffled = shuffle(rps_training_data, random_state=42)
rps_test_data_shuffled = shuffle(rps_test_data, random_state=42)

Now we perform feature scaling on the training and test data, as well as split the data into features and labels.

In [27]:
from sklearn.preprocessing import MinMaxScaler

feature_scaler = MinMaxScaler(feature_range=(0, 1)).set_output(transform="pandas")

rps_training_features = feature_scaler.fit_transform(rps_training_data_shuffled.drop("RPS Label", axis=1))
rps_training_labels = rps_training_data_shuffled[["RPS Label"]]

rps_test_features = feature_scaler.transform(rps_test_data_shuffled.drop("RPS Label", axis=1))
rps_test_labels = rps_test_data_shuffled[["RPS Label"]]

### Question 3
Train a binary classifier to distinguish between just two classes, paper, and non-paper
using `LogisticRegression`, `LinearSVC (C = 0.01)`, and `LinearSVC (C = 100)`. For each model:
1. Predict $\hat{y}$ for some new instances (`X_new`) where (`X_new = Xtrain[88], Xtrain[2159], Xtrain[1346]`).
2. Find the probability that the model used to predict the class of an instance (if applicable).
3. Calculate the score (using the test set)
4. What are the `decision_function` values?
5. Write your observation about the results from the table.

In [28]:
# set up X_new
X_new_features = rps_training_features.iloc[[88, 2159, 1346],:]
X_new_labels = rps_training_labels.iloc[[88, 2159, 1346],:]

#### `LogisticRegression`

Train the model.

In [29]:
import numpy as np
np.set_printoptions(suppress=True) # prevent numpy from printing in scientific notation
from sklearn.linear_model import LogisticRegression

log_reg = LogisticRegression(random_state=42)
log_reg.fit(rps_training_features, np.ravel(rps_training_labels))

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(random_state=42)

Find the probability that the model used to predict the class of an instance.

In [30]:
log_reg.predict_proba(X_new_features)

array([[0.        , 0.        , 1.        ],
       [0.00004499, 0.9999288 , 0.00002621],
       [0.00069741, 0.99930044, 0.00000215]])

Predict labels for `X_new`.

In [31]:
log_reg.predict(X_new_features)

array([3, 2, 2])

Score the logistic regression model.

In [32]:
log_reg.score(rps_test_features, rps_test_labels)

0.6263440860215054

Get values of the `decision_function` for `X_new`.

In [33]:
log_reg.decision_function(X_new_features)

array([[-7.71530141, -7.96697771, 15.68227912],
       [-3.15621806,  6.85277665, -3.69655859],
       [-0.49520806,  6.77223066, -6.2770226 ]])

#### `LinearSVC (C = 0.01)`

#### `LinearSVC (C = 100)`

#### Table for question 3:

| Model/Technique | Predict `X_new` | Probability | `decision_function` | Score
| --- | --- | --- | --- | --- |
| `LogisticRegression` |
| `LinearSVC (C = 0.01)` |
| `LinearSVC (C = 100)` |

### Question 4
How can you predict the class of an instance using:
1. Probability (if applicable)
2. `decision_function`

## Part 3


### Question 1
Use the same data (Rock-Paper-Scissor” dataset) in part 2 (training data and test data) to train multiclass classifiers using Softmax Regression, Polynomial Kernel SVM (degree=10, coef0=1, C=5), and Polynomial Kernel SVM with your suggested values.
For each model:
1. Predict $\hat{y}$ for some new instances (`X_new`) where (`X_new = Xtrain[88], Xtrain[2159], Xtrain[1346]`).
2. Find the probability that the model used to predict the class of an instance (if applicable).
3. Calculate the score (using the test set)
4. What are the `decision_function` values?
5. Write your observation about the results from the table.

In [34]:
# TODO

Table for question 1:

| Model/Technique | Predict `X_new` | Probability | `decision_function` | Score
| --- | --- | --- | --- | --- |
| `Softmax Regression` |
| `Polynomial Kernel SVM (degree=10, ceof0=1, C=5)` |
| `Polynomial Kernel SVM` **(your suggested values)** |

### Question 2
Train a grid search model to find the best combination to train SVM classifiers between `C` and `gamma` with `kernel = rbf` as follows:
```
para_grid = {'C': [0.1, 1, 10, 100, 1000],
            'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
            'kernel': ['rbf]}
```
1. What are the best values for the hyperparameters?
2. What is the best score?

In [35]:
# TODO